In [1]:
import os
import gc
import re
import sys
import math
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np
from xpinyin import Pinyin

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [95]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# reload(sys)
# sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
# PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
# PRODUCE_DB_USER = 'pingjia'
# PRODUCE_DB_PASSWD = 'De32wsxC'

PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'

PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://' + PRODUCE_DB_USER + ':' + PRODUCE_DB_PASSWD + '@' + PRODUCE_DB_ADDR_OUTTER + '/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE)

# query_sql = 'select * from pingjia.open_model_detail '
query_sql = 'select * from pingjia.open_category '
# query_sql = 'select source_id,detail_name,year,month,mile,price,city,domain,transfer_owner from crawler_car_source'
origin_train = pd.read_sql_query(query_sql, engine)
origin_train.to_csv('./origin_open_category.csv', index=False)
print('finish query, num:'+str(len(origin_train)))

finish query, num:2033


### 一.车系匹配

In [49]:
def process_brand_name(df):
    """
    删除没用的字符
    """
    text = df['brand_name']
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('MINI迷你', 'MINI')
    text = text.replace('RUF如虎', '如虎')
    text = text.replace('Pagani', '帕加尼')
    text = text.replace('柯尼赛格', '科尼赛克')
    text = text.replace('布嘉迪', '布加迪')
    text = text.replace('幻速', '北汽幻速')
    text = text.replace('昌河', '北汽昌河')
    text = text.replace('汽车', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

def process_model_name(df):
    """
    删除没用的字符
    """
    text = df['model_name']
    text = text.lower()
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('-', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('/', '')
    text = text.replace('°', '')
    text = text.replace('!', '')
    text = text.replace('ⅰ', 'i')
    text = text.replace('ⅱ', 'ii')
    text = text.replace('ⅲ', 'iii')
    text = text.replace('pagani', '帕加尼')
    text = text.replace('柯尼赛格', '科尼赛克')
    text = text.replace('科尼赛格', '科尼赛克')
    text = text.replace('布嘉迪', '布加迪')
    text = text.replace('吉利康迪全球鹰', '全球鹰')
    text = text.replace('汽车', '')
    if df['brand_name'] == '长丰猎豹':
        text = text.replace('猎豹', '长丰猎豹')
    if df['brand_name'] == '北汽制造':
        text = text.replace('北汽', '')
    if df['brand_name'] == '北汽绅宝':
        text = text.replace('北京绅宝', '北汽绅宝')
    if df['brand_name'] == '北汽幻速':
        text = text.replace('幻速', '北汽幻速')
    if df['brand_name'] == '北汽昌河':
        text = text.replace('昌河', '北汽昌河')
    text = text.replace(' ', '')
    # text = text.lower()
    return text

def process_final_text(df):
    """
    删除没用的字符
    """
    text = df['model_name'].replace(df['brand_name'], '')
    # text = text.lower()
    return df['brand_name']+text

In [51]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
open_category = open_category.loc[~(open_category['status'] == 'D'), :].reset_index(drop=True)

# 人工解决的车系问题
open_category.loc[(open_category['id'] == 204), 'name'] = '奔驰A级(进口)'
open_category = open_category.loc[~(open_category['id'].isin([1069,1264])), :] 

liyang_brand = open_category.loc[(open_category['parent'].isnull()), ['id', 'name', 'slug']].rename(columns={'name': 'brand_name', 'slug': 'parent'}).reset_index(drop=True)
liyang_model = open_category.loc[(open_category['parent'].notnull()), ['id', 'name', 'slug', 'parent']].rename(columns={'name': 'model_name',
             'slug': 'model_slug', 'id': 'model_id'}).reset_index(drop=True)
liyang_model = liyang_model.merge(liyang_brand, how='left', on=['parent']).rename(columns={'id': 'brand_id'})
liyang_model = liyang_model.loc[:, ['brand_id', 'model_id', 'brand_name', 'model_name','parent', 'model_slug']]

liyang_model['brand_name'] = liyang_model.apply(process_brand_name, axis=1)
liyang_model['model_name'] = liyang_model.apply(process_model_name, axis=1)
liyang_model['final_text'] = liyang_model.apply(process_final_text, axis=1)
liyang_model = liyang_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
liyang_model = liyang_model.rename(columns={'brand_id':'liyang_brand_id','model_id':'liyang_model_id','parent':'brand_slug',
                                                       'brand_name':'liyang_brand_name','model_name':'liyang_model_name'})
print('力洋车系数量:',len(liyang_model))

verify = liyang_model.groupby(['final_text'])['model_slug'].count().reset_index().rename(columns={'model_slug':'num'})
print('车系名称重复的车系,请人工检查:')
print(verify.loc[(verify['num'] > 1),:])
liyang_model.to_csv('./man_liyang.csv', index=False)

力洋车系数量: 1763
车系名称重复的车系,请人工检查:
Empty DataFrame
Columns: [final_text, num]
Index: []


In [52]:
def process_brand_name(df):
    """
    删除没用的字符
    """
    text = df['brand_name']

    text = text.replace('Jeep', '吉普')
    text = text.replace('SWM斯威汽车', '斯威')
    text = text.replace('名爵', 'MG')
    text = text.replace('上汽大通', '大通')
    text = text.replace('福汽启腾', '启腾')
    text = text.replace('汽车', '')
    text = text.replace('・', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

def process_model_name(df):
    """
    删除没用的字符
    """
    text = df['model_name']
    text = text.lower()
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('-', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('/', '')
    text = text.replace('°', '')
    text = text.replace('!', '')
    text = text.replace('名爵', 'mg')
    text = text.replace('上汽大通', '大通')
    text = text.replace('风行', '东风风行')
    text = text.replace('swm斯威', '斯威')
    if df['brand_name'] == '北汽制造':
        text = text.replace('北汽', '')
    if df['brand_name'] == '北汽绅宝':
        text = text.replace('绅宝', '北汽绅宝')
    text = text.replace('汽车', '')
    text = text.replace(' ', '')

    return text

def process_final_text(df):
    """
    删除没用的字符
    """
    text = df['model_name'].replace(df['brand_name'], '')
    # text = text.lower()
    return df['brand_name']+text

In [53]:
car_autohome_all = pd.read_csv('./car_autohome_all.csv')
car_autohome_model = car_autohome_all.loc[:, ['brand_slug', 'model_slug', 'brand_name', 'model_name']]
car_autohome_model = car_autohome_model.drop_duplicates(['brand_slug', 'model_slug']).reset_index(drop=True)
car_autohome_model['brand_name'] = car_autohome_model.apply(process_brand_name, axis=1)
car_autohome_model['model_name'] = car_autohome_model.apply(process_model_name, axis=1)
car_autohome_model['final_text'] = car_autohome_model.apply(process_final_text, axis=1)
car_autohome_model = car_autohome_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
car_autohome_model = car_autohome_model.rename(columns={'brand_slug':'car_autohome_brand_id','model_slug':'car_autohome_model_id',
                                                       'brand_name':'car_autohome_brand_name','model_name':'car_autohome_model_name'})
print('汽车之家车系数量:',len(car_autohome_model))

verify = car_autohome_model.groupby(['final_text'])['car_autohome_model_id'].count().reset_index().rename(columns={'car_autohome_model_id':'num'})
print('车系名称重复的车系,请人工检查:')
print(verify.loc[(verify['num'] > 1),:])

car_autohome_model.to_csv('./man_car_autohome.csv', index=False)

汽车之家车系数量: 1880
车系名称重复的车系,请人工检查:
Empty DataFrame
Columns: [final_text, num]
Index: []


#### 车系匹配

In [122]:
origin_combine_model = pd.read_csv('./origin_open_category.csv')
combine_model = pd.read_csv('../tmp/train/combine_model.csv')
origin_combine_model = origin_combine_model.merge(combine_model.loc[:,['slug','car_autohome_model_id']],how='left',on=['slug'])

liyang_brand = origin_combine_model.loc[(origin_combine_model['parent'].isnull()), ['id', 'name', 'slug']].rename(columns={'name': 'brand_name', 'slug': 'parent'}).reset_index(drop=True)
liyang_model = origin_combine_model.loc[(origin_combine_model['parent'].notnull()), ['id', 'name', 'slug', 'parent','car_autohome_model_id']].rename(columns={'name': 'model_name',
             'slug': 'model_slug', 'id': 'model_id'}).reset_index(drop=True)
liyang_model = liyang_model.merge(liyang_brand, how='left', on=['parent']).rename(columns={'id': 'brand_id'})
liyang_model = liyang_model.loc[:, ['brand_id', 'model_id', 'brand_name', 'model_name','parent', 'model_slug','car_autohome_model_id']]
liyang_model = liyang_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
liyang_model = liyang_model.rename(columns={'brand_id':'liyang_brand_id','model_id':'liyang_model_id','parent':'brand_slug',
                                                       'brand_name':'liyang_brand_name','model_name':'liyang_model_name'})

match_result = car_autohome_model.copy()
match_result = match_result.merge(liyang_model.loc[:, ['liyang_brand_id','liyang_model_id','brand_slug','model_slug','car_autohome_model_id']], how='left', on=['car_autohome_model_id'])

step1 = match_result.loc[(match_result['liyang_model_id'].notnull()),:].reset_index(drop=True)
step2 = match_result.loc[(match_result['liyang_model_id'].isnull()),:].reset_index(drop=True)
print('总数量:',len(match_result),'匹配数量:',len(step1),'未匹配数量:',len(step2))

总数量: 1880 匹配数量: 1851 未匹配数量: 29


In [123]:
def process_brand_slug(df):
    if type(df['brand_slug']) == float:
        temp = temp_match.loc[(temp_match['car_autohome_brand_name'] == df['car_autohome_brand_name'])&(temp_match['liyang_brand_id'].notnull()),:].reset_index(drop=True)
        if len(temp) > 0:
            liyang_brand_id = temp.loc[0,'liyang_brand_id']
            brand_slug = temp.loc[0,'brand_slug']
        else:
            liyang_brand_id = np.NaN
            brand_slug = 'car_autohome_brand_'+str(df['car_autohome_brand_id'])
    else:
        liyang_brand_id = df['liyang_brand_id']
        brand_slug = df['brand_slug']
    if type(df['model_slug']) == float:
        model_slug = 'car_autohome_model_'+str(df['car_autohome_model_id'])
    else:
        model_slug = df['model_slug']
    return pd.Series([liyang_brand_id,brand_slug,model_slug])


brand_count = match_result.loc[:,['car_autohome_brand_id','brand_slug']].copy()
brand_count = brand_count.drop_duplicates(['car_autohome_brand_id','brand_slug'])
brand_count = brand_count.groupby(['brand_slug'])['car_autohome_brand_id'].count().reset_index()
brand_count = brand_count.loc[(brand_count['car_autohome_brand_id'] > 1),:]
match_result.loc[(match_result['brand_slug'].isin(list(set(brand_count.brand_slug.values)))),['liyang_brand_id','brand_slug']] = np.NaN,np.NaN
temp_match = match_result.copy()
match_result[['liyang_brand_id','brand_slug','model_slug']] = match_result.apply(process_brand_slug,axis=1)
match_result.to_csv('./man_liyang_model_match_result.csv',index=False)
print(match_result.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1880 entries, 0 to 1879
Data columns (total 9 columns):
car_autohome_brand_id      1880 non-null int64
car_autohome_model_id      1880 non-null int64
car_autohome_brand_name    1880 non-null object
car_autohome_model_name    1880 non-null object
final_text                 1880 non-null object
liyang_brand_id            1531 non-null float64
liyang_model_id            1851 non-null float64
brand_slug                 1880 non-null object
model_slug                 1880 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 146.9+ KB
None


### 二.款型匹配

In [125]:
def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """
    text = df[column_name]
    text = text.lower()
    text = text.replace('ⅰ', 'i')
    text = text.replace('ⅱ', 'ii')
    text = text.replace('ⅲ', 'iii')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace('【', '')
    text = text.replace('】', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('_', '')
    text = text.replace('-', '')
    text = text.replace('+', '')
    text = text.replace('—', '')
    text = text.replace('/', '')
    text = text.replace('“', '')
    text = text.replace('”', '')
    text = text.replace('!', '')
    text = text.replace('。', '')
    text = text.replace('＞', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('》', '')
    text = text.replace('！', '')
    text = text.replace('／', '')
    text = text.replace('’', '')
    text = text.replace('－', '')
    text = text.replace('•', '')
    text = text.replace('×', '')
    text = text.replace('《', '')
    text = text.replace('＿', '')
    text = text.replace('®', '')
    text = text.replace('─', '')
    text = text.replace('、', '')
    text = text.replace(' ', '')
    return text

temp_match = match_result.loc[:,['car_autohome_model_id','liyang_brand_id','liyang_model_id','brand_slug','model_slug']].rename(columns={'model_slug':'liyang_model_slug','brand_slug':'liyang_brand_slug','car_autohome_model_id':'model_slug'})
open_model_detail = pd.read_csv('./origin_open_model_detail.csv',low_memory=False)
open_model_detail = open_model_detail.loc[(open_model_detail['status'] != 'D'),:].reset_index(drop=True)

open_model_detail = open_model_detail.loc[:,['global_slug','detail_model','detail_model_slug','year','volume','control','price_bn']].rename(columns={'detail_model_slug':'model_detail_slug'})
open_model_detail['control'] = open_model_detail['control'].fillna('自动')
open_model_detail['final_text'] = open_model_detail.apply(delete_str_useless,args=('detail_model',),axis=1)
car_autohome_all = pd.read_csv('./car_autohome_all.csv',low_memory=False)
car_autohome_all = car_autohome_all.merge(temp_match,how='left',on=['model_slug'])
car_autohome_all['final_text'] = car_autohome_all.apply(delete_str_useless,args=('detail_name',),axis=1)

#### 1.生成词向量映射

In [126]:
liyang_final_text = open_model_detail.copy()
car_autohome_final_text = car_autohome_all.copy()

texts = []
train_final = []

for i in range(0, len(liyang_final_text)):
    temp = []
    texts.extend(liyang_final_text['final_text'][i])
    x = liyang_final_text['final_text'][i]
    temp.extend(x)
    train_final.append(temp)
    
for i in range(0, len(car_autohome_final_text)):
    temp = []
    texts.extend(car_autohome_final_text['final_text'][i])
    x = car_autohome_final_text['final_text'][i]
    temp.extend(x)
    train_final.append(temp)

# 创建Tokenizer
tokenizer = Tokenizer(num_words=len(set(texts)), char_level=True, lower=True)
tokenizer.fit_on_texts(train_final)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 955 unique tokens


In [127]:
####################################### 力洋
liyang_final_text['fill_vector'] = np.NaN

train_final = []
for i in range(0, len(liyang_final_text)):
    temp = []
    x = liyang_final_text['final_text'][i]
    temp.extend(x)
    train_final.append(temp)
train_sequences = tokenizer.texts_to_sequences(train_final)

fill_zero = [0 for i in range(0, len(word_index))]

for i in range(0, len(liyang_final_text)):
    text = fill_zero.copy()
    for j in train_sequences[i]:
        text[j - 1] = text[j - 1] + 1
    liyang_final_text.loc[i, 'fill_vector'] = str(text)

####################################### 汽车之家
car_autohome_final_text['fill_vector'] = np.NaN

train_final = []
for i in range(0, len(car_autohome_final_text)):
    temp = []
    x = car_autohome_final_text['final_text'][i]
    temp.extend(x)
    train_final.append(temp)
train_sequences = tokenizer.texts_to_sequences(train_final)

fill_zero = [0 for i in range(0, len(word_index))]

for i in range(0, len(car_autohome_final_text)):
    text = fill_zero.copy()
    for j in train_sequences[i]:
        text[j - 1] = text[j - 1] + 1
    car_autohome_final_text.loc[i, 'fill_vector'] = str(text)

In [128]:
def cal_cos_similar(df,y_vector):
    x_vector = eval(df['fill_vector'])
    y_vector = eval(y_vector)
    return np.dot(x_vector,y_vector) / (np.linalg.norm(x_vector,ord=2)*np.linalg.norm(y_vector,ord=2) )

def match_detail(df):
    print(df['model_name'],df['detail_name'])
    if str(df['liyang_model_id']) == 'nan':
        return pd.Series([np.NaN,np.NaN,np.NaN,np.NaN]) 
    
    temp_detail = liyang_final_text.loc[(liyang_final_text['global_slug'] == df['liyang_model_slug'])&(liyang_final_text['year'] == df['online_year'])&(liyang_final_text['control'] == df['control'])&(liyang_final_text['volume'] == df['volume']),:].reset_index(drop=True)

    if len(temp_detail) == 0:
        return pd.Series([np.NaN,np.NaN,np.NaN,np.NaN])
    
    temp_detail['cos_similar'] = temp_detail.apply(cal_cos_similar,args=(df['fill_vector'],),axis=1)
    result = temp_detail.loc[temp_detail.groupby(['global_slug']).cos_similar.idxmax(), ['detail_model','model_detail_slug','price_bn','cos_similar']].values[0]
    return pd.Series(result)

car_autohome_final = car_autohome_final_text.copy()
car_autohome_final[['liyang_detail_model','model_detail_slug','liyang_price_bn','cos_similar']] = car_autohome_final.apply(match_detail,axis=1) 
car_autohome_final.to_csv('./man.csv',index=False)

宝来/宝来经典 2004款 1.6 手动基本
宝来/宝来经典 2004款 1.6 手动基本
宝来/宝来经典 2004款 1.6 自动基本
宝来/宝来经典 2004款 1.6 自动天窗
宝来/宝来经典 2004款 1.8 手动尊贵
宝来/宝来经典 2004款 1.8 自动尊贵
宝来/宝来经典 2004款 1.8 手动豪华
宝来/宝来经典 2004款 1.8 自动舒适
宝来/宝来经典 2004款 1.8 手动舒适
宝来/宝来经典 2004款 1.8T 自动豪华
宝来/宝来经典 2004款 1.8T 手动舒适
宝来/宝来经典 2004款 1.8T 手动尊贵
宝来/宝来经典 2004款 1.8T 手动豪华
宝来/宝来经典 2004款 1.8T 自动尊贵
宝来/宝来经典 2004款 1.6 手动天窗
宝来/宝来经典 2004款 1.8 自动豪华
宝来/宝来经典 2004款 1.9 TDI
宝来/宝来经典 2005款 1.6手动基本
宝来/宝来经典 2005款 1.6手动天窗
宝来/宝来经典 2005款 1.6自动基本
宝来/宝来经典 2005款 1.6自动天窗
宝来/宝来经典 2005款 1.8手动舒适
宝来/宝来经典 2005款 1.8T R(运动版)
宝来/宝来经典 2005款 1.8自动舒适
宝来/宝来经典 2005款 1.8自舒天窗
宝来/宝来经典 2005款 1.8奥运版
宝来/宝来经典 2006款 1.6 手动时尚 TL
宝来/宝来经典 2006款 1.8 手动豪华 HL
宝来/宝来经典 2006款 1.6 自动时尚 TL
宝来/宝来经典 2006款 1.6 手动豪华 HL
宝来/宝来经典 2006款 1.6 自动豪华 HL
宝来/宝来经典 2006款 1.8 自动豪华 HL
宝来/宝来经典 2006款 HS 1.6 自动豪华
宝来/宝来经典 2006款 HS 1.6 手动时尚
宝来/宝来经典 2006款 HS 1.6 自动时尚
宝来/宝来经典 2006款 HS 1.6 手动豪华
宝来/宝来经典 2006款 HS 1.8 手动豪华
宝来/宝来经典 2006款 HS 1.8 自动豪华
捷达 2005款 CIF 基本型
捷达 2005款 GIF 前卫豪华型
捷达 2008款 CIF 春天型
捷达 2008款 CIF+AT 春天型
捷达 2004款 GDF 豪华型
捷达

辉腾 2005款 3.2L V6 5座豪华版
辉腾 2005款 6.0L W12 5座豪华版
辉腾 2006款 4.2L V8 4座豪华版
辉腾 2006款 6.0L W12 4座豪华版
辉腾 2007款 3.2L V6 5座基本版
辉腾 2009款 3.6L V6 5座加长舒适版
辉腾 2009款 3.6L V6 5座加长顶级版
辉腾 2009款 3.6L V6 4座加长行政版
辉腾 2009款 3.6L V6 5座加长豪华版
辉腾 2009款 4.2L V8 5座加长行政版
辉腾 2009款 4.2L V8 4座加长行政版
辉腾 2009款 6.0L W12 5座加长尊崇版
辉腾 2009款 6.0L W12 4座加长尊崇版
辉腾 2011款 3.6L V6 5座加长商务版
辉腾 2011款 6.0L W12 4座加长Individual版
辉腾 2011款 4.2L V8 4座加长Individual版
辉腾 2012款 3.0TDI 行政型
辉腾 2011款 3.6L V6 5座加长舒适版
辉腾 2011款 3.6L V6 5座加长Individual版
辉腾 2011款 3.6L V6 4座加长Individual版
辉腾 2012款 3.0L 商务型
辉腾 2012款 3.0L 行政型
辉腾 2012款 3.0L 精英定制型
辉腾 2012款 3.0TDI 精英定制型
辉腾 2012款 3.6L 尊享定制型
辉腾 2012款 4.2L 奢享定制型
辉腾 2012款 6.0L 奢享定制型
辉腾 2013款 4.2L 柏秋纳弗洛版
辉腾 2014款 3.0L 商务型
辉腾 2014款 3.0L 行政型
辉腾 2015款  4.2L 奢享版
辉腾 2015款 3.0L 智享版
辉腾 2015款 3.0L 专享版
辉腾 2015款 3.6L 尊享版
途安 2004款 2.0L 手豪5座
途安 2004款 2.0L 自豪5座
途安 2004款 2.0L 手豪7座
途安 2004款 2.0L 自豪7座
途安 2004款 2.0L 手基5座
途安 2005款 1.8T 手舒5座
途安 2005款 1.8T 自豪5座
途安 2005款 1.8T 手舒7座
途安 2005款 1.8T 自豪7座
途安 2005款 2.0L 手舒5座
途安 2005款 1.8T 自舒5座
途

帕萨特 2003款 2.0L 手动基本型
帕萨特 2003款 2.0L 自动基本型
帕萨特 2005款 2.0L 手动超值版
帕萨特 2005款 1.8T 手动增值版
帕萨特 2005款 2.0L 自动超值版
帕萨特 2007款 1.8T 手动舒适型
帕萨特 2007款 2.0L 手动基本型
帕萨特 2007款 2.0L 自动基本型
帕萨特 2005款 1.8T 自动豪华增值版
帕萨特 2011款 2.0TSI DSG至尊版
帕萨特 2011款 1.8TSI DSG尊荣版
帕萨特 2011款 1.8TSI DSG御尊版
帕萨特 2011款 1.8TSI DSG至尊版
帕萨特 2011款 2.0TSI DSG御尊版
帕萨特 2011款 3.0L V6 DSG旗舰版
帕萨特 2011款 1.4TSI DSG尊荣版
帕萨特 2011款 1.4TSI 手动尊荣版
帕萨特 2011款 3.0L V6 DSG旗舰尊享版
帕萨特 2011款 1.8TSI 自动尊荣版
帕萨特 2013款 1.4TSI DSG蓝驱版
帕萨特 2013款 3.0L V6 DSG旗舰尊享版
帕萨特 2013款 1.4TSI 手动尊雅版
帕萨特 2013款 1.4TSI 手动尊荣版
帕萨特 2013款 1.4TSI DSG尊荣版
帕萨特 2013款 1.8TSI DSG尊荣版
帕萨特 2013款 1.8TSI DSG御尊版
帕萨特 2013款 1.8TSI DSG至尊版
帕萨特 2013款 2.0TSI DSG御尊版
帕萨特 2013款 2.0TSI DSG至尊版
帕萨特 2013款 3.0L V6 DSG旗舰版
帕萨特 2013款 1.8TSI DSG政府采购版
帕萨特 2014款 1.4TSI 手动尊雅版
帕萨特 2014款 1.4TSI 手动尊荣版
帕萨特 2014款 1.4TSI DSG尊荣版
帕萨特 2014款 1.4TSI DSG蓝驱技术版
帕萨特 2014款 1.8TSI DSG尊雅版
帕萨特 2014款 1.8TSI DSG尊荣版
帕萨特 2014款 1.8TSI DSG尊荣导航版
帕萨特 2014款 1.8TSI DSG御尊版
帕萨特 2014款 1.8TSI DSG御尊导航版
帕萨特 2014款 1.8TSI DSG至尊版
帕萨特 2014款 2.0TSI DSG御尊版
帕萨特 201

Passat领驭 2011款 2.0L 手动尊享型
Passat领驭 2011款 1.8T 手动尊享型
Passat领驭 2011款 1.8T 自动尊享型
Passat领驭 2011款 1.8T 自动尊品型
Passat领驭 2011款 2.0L 手动尊品型
高尔夫 2005款 1.8 手动舒适型
高尔夫 2005款 1.8 自动舒适型
高尔夫 2004款 1.6 手动舒适型
高尔夫 2004款 1.6 自动舒适型
高尔夫 2004款 1.6 手动时尚型
高尔夫 2004款 1.6 自动时尚型
高尔夫 2008款 1.6 手动五彩版
高尔夫 2008款 1.6 自动五彩版
高尔夫 2010款 1.4TSI 自动豪华型
高尔夫 2010款 1.6 手动时尚型
高尔夫 2010款 1.6 自动时尚型
高尔夫 2010款 1.6 自动舒适型
高尔夫 2010款 1.4TSI 手动舒适型
高尔夫 2010款 1.4TSI 自动舒适型
高尔夫 2010款 1.6 手动舒适型
高尔夫 2010款 1.6 自动豪华版
高尔夫 2010款 2.0TSI GTI
高尔夫 2008款 1.6 自动时尚型
高尔夫 2012款 1.4TSI 蓝驱版
高尔夫 2011款 1.6 手动时尚型
高尔夫 2011款 1.6 自动时尚型
高尔夫 2011款 1.6 手动舒适型
高尔夫 2011款 1.6 自动舒适型
高尔夫 2011款 1.6 自动豪华版
高尔夫 2011款 1.4TSI 手动舒适型
高尔夫 2011款 1.4TSI 自动舒适型
高尔夫 2011款 1.4TSI 自动豪华型
高尔夫 2011款 2.0TSI GTI
高尔夫 2012款 1.4TSI 自动豪华型
高尔夫 2012款 1.4TSI 自动舒适型
高尔夫 2012款 1.6 手动时尚型
高尔夫 2012款 1.6 自动时尚型
高尔夫 2012款 1.6 手动舒适型
高尔夫 2012款 1.6 自动舒适型
高尔夫 2012款 1.6 自动豪华版
高尔夫 2012款 1.4TSI 手动舒适型
高尔夫 2012款 2.0TSI GTI
高尔夫 2014款 1.4TSI 自动旗舰型
高尔夫 2014款 1.6L 手动时尚型
高尔夫 2014款 1.6L 自动时尚型
高尔夫 2014款 1.6L 自动舒适型
高尔夫 2014款 1.4

辉昂 2016款 480 V6 四驱行政旗舰版
辉昂 2016款 380TSI 两驱御尊版
辉昂 2016款 380TSI 两驱至尊版
辉昂 2016款 480 V6 四驱至尊版
辉昂 2016款 380TSI 两驱御尊豪华版
辉昂 2016款 380TSI 两驱行政版
辉昂 2016款 380TSI 两驱行政旗舰版
辉昂 2016款 480 V6 四驱行政版
辉昂 2018款 380TSI 两驱商务版
辉昂 2018款 380TSI 两驱豪华版
辉昂 2018款 380TSI 两驱旗舰版
辉昂 2018款 380TSI 两驱至尊旗舰版
辉昂 2018款 480 V6 四驱豪华版
辉昂 2018款 480 V6 四驱旗舰版
辉昂 2018款 480 V6 四驱至尊旗舰版
辉昂 2018款 改款 380TSI 两驱商务版
辉昂 2018款 改款 380TSI 两驱豪华版
辉昂 2018款 改款 380TSI 两驱旗舰版
辉昂 2018款 改款 380TSI 两驱豪华旗舰版
辉昂 2018款 改款 480 V6 四驱豪华版
辉昂 2018款 改款 480 V6 四驱旗舰版
辉昂 2018款 改款 480 V6 四驱豪华旗舰版
C-TREK蔚领 2017款 230TSI DSG豪华型
C-TREK蔚领 2017款 1.6L 手动舒适型
C-TREK蔚领 2017款 1.6L 自动舒适型
C-TREK蔚领 2017款 1.6L 手动豪华型
C-TREK蔚领 2017款 1.6L 自动豪华型
C-TREK蔚领 2017款 230TSI 手动舒适型
C-TREK蔚领 2017款 230TSI DSG舒适型
C-TREK蔚领 2018款 1.5L 手动舒适型
C-TREK蔚领 2018款 1.5L 自动舒适型
C-TREK蔚领 2018款 1.5L 手动豪华型
C-TREK蔚领 2018款 1.5L 自动豪华型
C-TREK蔚领 2018款 230TSI 手动舒适型
C-TREK蔚领 2018款 230TSI DSG舒适型
C-TREK蔚领 2018款 230TSI DSG豪华型
C-TREK蔚领 2018款 1.5L 手动时尚型
C-TREK蔚领 2018款 1.5L 自动时尚型
途昂 2017款 530 V6 四驱至尊旗舰版
途昂 2017款 330TSI 两驱豪华版
途昂 

...

神骐F30 2018款 1.5L单排舒适版2.7米货厢1015载重DAM15L
神骐F30 2018款 1.5L单排标准版2.7米货厢495载重DAM15L
神骐F30 2018款 1.5L单排标准版2.7米货厢745载重DAM15L
神骐F30 2018款 1.5L单排标准版2.7米货厢1015载重DAM15L
神骐F50 2016款 2.2L豪华版长轴491QE
神骐F50 2016款 2.2L豪华版短轴491QE
神骐F50 2016款 2.2L舒适版长轴491QE
神骐F50 2016款 2.2L舒适版短轴491QE
睿行S50 2017款 1.5L 手动旗舰型
睿行S50 2017款 1.5L 手动舒适型
睿行S50 2017款 1.5T 手动旗舰型 6MT
睿行S50 2017款 1.5L 手动经济型
睿行S50 2017款 1.5T 手动标准型 6MT
睿行S50 2017款 1.5T 手动舒适型 6MT
睿行S50 2017款 1.5L 手动标准型
睿行S50 2017款 1.5L 手动豪华型
睿行S50 2017款 1.5T 手动豪华型 5MT
睿行S50 2017款 1.5T 手动标准型 5MT
睿行S50 2017款 1.5T 手动舒适型 5MT
睿行S50 2017款 1.5T 手动旗舰型 5MT
睿行S50 2017款 1.5T 手动豪华型 6MT
睿行S50 2017款 S50V 1.5L 手动物流版Ⅲ
睿行S50 2017款 S50V 1.5L 手动物流版Ⅱ
睿行S50 2017款 S50V 1.5L 手动物流版I
睿行S50 2018款 S50V 1.5L 物流版5座基础型
睿行S50 2018款 S50V 1.5L 物流版5座
睿行S50 2018款 S50V 1.5L 物流版7座
睿行S50T 2018款 1.5L 手动标准版
睿行S50T 2018款 1.5T 手动精英版
睿行S50T 2018款 1.5T 手动旗舰版
睿行M70 2017款 1.5L标准型对开门DAM15DL
睿行M70 2017款 1.5L舒适型对开门DAM15DL
睿行M70 2017款 1.5L舒适型背掀门DAM15DL
睿行M70 2017款 1.5L标准型背掀门DAM15DL
睿行M70 2017款 1.5L舒适型7座对开门DAM15DL
睿行M70

奥迪A5 2019款 Coupe 45 TFSI quattro 运动型
奥迪A5 2019款 Cabriolet 40 TFSI 时尚型
奥迪A5 2019款 Cabriolet 45 TFSI quattro 运动型
奥迪A7 2019款 55 TFSI quattro 动感型
奥迪A7 2019款 55 TFSI quattro 竞技版
奥迪RS 4 2018款  RS 4 2.9T Avant
奥迪Q7新能源 2019款 55 e-tron
迈巴赫S级 2019款 S 680 双调典藏版
奔驰GLE 2019款 GLE 320 4MATIC 动感型臻藏版
奔驰GLE 2019款 GLE 300 d 4MATIC
奔驰GLE 2019款 GLE 320 4MATIC 豪华型臻藏版
奔驰GLE 2019款 GLE 350 d 4MATIC
奔驰GLE 2019款 GLE 400 4MATIC 臻藏版
奔驰A级 2019款 A 200 L 运动轿车
奔驰A级 2019款 A 200 L
奔驰A级 2019款 A 200 L 运动轿车先行特别版
奔驰A级 2019款 A 180 L
奔驰A级 2019款 A 180 L 运动轿车
别克GL8 2018款 ES 28T 舒适型 国VI
别克GL8 2018款 ES 28T 尊享型 国VI
别克GL8 2018款 ES 28T 豪华型 国VI
别克GL8 2018款 ES 28T 旗舰型 国VI
别克GL8 2018款 28T Avenir 国VI
别克GL8 2018款 28T 豪华型 国VI
别克GL8 2018款 28T 尊贵型 国VI
英朗 2019款 15T 双离合互联精英型
英朗 2019款 18T 自动互联精英型 国VI
英朗 2019款 18T 自动互联旗舰型 国VI
昂科威 2019款 20T 两驱领先型 国VI
昂科威 2019款 20T 两驱精英型 国VI
昂科威 2019款 20T 两驱豪华型 国VI
昂科威 2019款 28T 四驱精英型 国VI
昂科威 2019款 28T 四驱豪华型 国VI
昂科威 2019款 28T 四驱全能运动旗舰型 国VI
别克GL6 2019款 18T 5座精英型 国VI
别克GL6 2019款 18T 6座精英型 国VI
别克GL6 2019款 18T 5座互联豪华

宝骏530 2019款 1.5T CVT尊贵型
宝骏530 2019款 1.5T CVT精英型
宝骏530 2019款 1.5T CVT豪华型
启辰T60 2018款 1.6L 手动时尚版
启辰T60 2018款 1.6L 手动智酷版
启辰T60 2018款 1.6L CVT智趣版
启辰T60 2018款 1.6L CVT智炫版
启辰T60 2018款 1.6L CVT智悦版
启辰T60 2018款 1.6L 手动智尚版
启辰T60 2018款 1.6L CVT智型版
雄师F16 2018款 2.8T柴油四驱进取型标双CA4D28C5-1B
雄师F16 2018款 2.8T柴油四驱进取型大双CA4D28C5-1B
依维柯Daily(欧胜) 2018款 3.0T短轴高顶多功能车F1C
依维柯Daily(欧胜) 2018款 2.3T短轴高顶多功能车F1A
依维柯Daily(欧胜) 2018款 3.0T自动客车14座5999长F1C
依维柯Daily(欧胜) 2018款 3.0T自动客车13座5999长F1C
依维柯Daily(欧胜) 2018款 3.0T自动客车19座7080长F1C
依维柯Daily(欧胜) 2018款 3.0T自动客车20座7080长F1C
依维柯Daily(欧胜) 2018款 3.0T长轴短悬多功能车F1C
依维柯Daily(欧胜) 2018款 2.3T短轴高顶厢式车F1A
依维柯Daily(欧胜) 2018款 3.0T短轴高顶厢式车F1C
依维柯Daily(欧胜) 2018款 3.0T短轴低顶厢式车F1C
依维柯Daily(欧胜) 2018款 3.0T长轴短悬厢式车F1C
依维柯Daily(欧胜) 2018款 2.3T长轴短悬厢式车F1A
依维柯Daily(欧胜) 2018款 3.0T自动长轴厢式车F1C
依维柯Daily(欧胜) 2018款 3.0T长轴厢式车F1C
依维柯Daily(欧胜) 2018款 3.0T自动加长轴单排厢式车F1C
依维柯Daily(欧胜) 2018款 3.0T自动加长轴双排厢式车F1C
依维柯Daily(欧胜) 2018款 3.0T加长轴双排厢式车F1C
观致5 2019款 1.6T 自动 智云互联（AliOS）版旗舰版
观致5 2019款 1.6T 自动 智云互联（AliOS）版尊贵版
上汽大通G50 2019款 1

君威 2019款 GS 28T 尊贵型 国V
君威 2019款 20T 领先型 国VI
君威 2019款 20T 精英型 国VI
君威 2019款 20T 豪华型 国VI
君威 2019款 20T 尊贵型 国VI
君威 2019款 28T 尊享型 国VI
君威 2019款 GS 28T 豪华型 国VI
君威 2019款 GS 28T 尊贵型 国VI
昂科威 2019款 20T 两驱领先型 国V
昂科威 2019款 20T 两驱精英型 国V
昂科威 2019款 20T 两驱豪华型 国V
昂科威 2019款 28T 四驱精英型 国V
昂科威 2019款 28T 四驱豪华型 国V
昂科威 2019款 28T 四驱全能运动旗舰型 国V
威朗 2019款 两厢 15S 自动进取型
威朗 2019款 两厢 15S 自动领先型
威朗 2019款 两厢GS 20T 双离合进取型
威朗 2019款 两厢GS 20T 双离合领先型
威朗 2019款 两厢GS 20T 双离合豪华型
欧陆 2019款 6.0T GT W12 敞篷版
保时捷911 2019款 Carrera 4S Cabriolet 3.0T
保时捷911 2019款 Carrera S Cabriolet 3.0T
牧马人 2019款 2.0T Sahara 四门炫顶版
指南者 2019款 200T 手动劲享-互联大屏版
指南者 2019款 200T 自动驭享-互联大屏版
指南者 2019款 200T 自动舒享-互联大屏版
指南者 2019款 200T 自动悦享-互联大屏版
指南者 2019款 200T 自动家享-互联大屏版
指南者 2019款 200T 自动新春特别版
指南者 2019款 200T 自动臻享-互联大屏版
指南者 2019款 200T 自动家享四驱-互联大屏版
指南者 2019款 200T 自动臻享四驱-互联大屏版
指南者 2019款 200TS 自动高性能四驱-互联大屏版
凯迪拉克CT6 2019款 40T 铂金版
凯迪拉克CT6 2019款 40T 豪华版
Huracán 2019款 Huracán EVO
Huracán 2019款 Huracán EVO Spyder
揽胜 2019款 5.0 V8 SC SVA巅峰创世加长版
发现神行 2019款 200PS PURE版
发现神行 2019款 2

欧尚X70A 2018款 1.3T 手动豪华型
欧尚X70A 2018款 1.3T 手动旗舰型
欧尚长行 2019款 1.5L 手动智联尊享型
欧尚长行 2019款 1.6L 自动智联尊贵型
风行SX6 2019款 1.5T 手动豪华型
风行SX6 2019款 1.5T 自动豪华型
风行SX6 2019款 1.6L 手动舒适型
风行SX6 2019款 1.6L 手动豪华型
风行SX6 2019款 1.6L CVT豪华型
金旅海狮新能源 2019款 龙运GLE800系列XML5036XXYEVW01
金旅海狮新能源 2019款 龙运GLE800系列XML5036XXYEVL01
金旅海狮新能源 2019款 龙运GLE800系列XML5036XXYEVL02
金旅海狮新能源 2019款 龙运GLE800系列XML5036XXYEVL03
哈弗H7 2019款 2.0T 智享型
哈弗H7 2019款 H7L 2.0T 豪华型
哈弗H7 2019款 2.0T 畅享型
哈弗H7 2019款 2.0T 悦享型
哈弗H7 2019款 2.0T 尊享型
哈弗H7 2019款 H7L 2.0T 尊贵型
哈弗H6 Coupe 2019款 1.5T 自动两驱豪华智联版
哈弗H6 Coupe 2019款 1.5T 自动两驱超豪智联版
哈弗H6 Coupe 2019款 1.5T 自动两驱都市型
哈弗H6 Coupe 2019款 1.5T 自动两驱精英型
路易斯 2019款 2.0T 自动两驱豪华版
路易斯 2019款 2.0T 自动两驱旗舰型
EC系列 2018款 EC220 标准版
北汽新能源EX5 2019款 悦风版
北汽新能源EX5 2019款 悦尚版
北汽新能源EX5 2019款 悦潮版
北汽新能源EX5 2019款 悦领版
骐铃T5 2019款 2.2T PLUS尊贵版标准轴距V22
骐铃T5 2019款 2.8T PLUS尊贵版标准轴距JE493ZLQ5D
骐铃T5 2019款 2.2T PLUS尊贵版加长轴距V22
骐铃T5 2019款 2.8T PLUS尊贵版加长轴距JE493ZLQ5D
骐铃T5 2019款 2.2T PLUS舒适版标准轴距V22
骐铃T5 2019款 2.8T PLUS舒适版标准轴距JE493ZLQ5D
骐铃T5 2019款 2.2T PLUS舒适版加长轴

In [129]:
def process_detail_slug(df):
    if type(df['model_detail_slug']) == float:
        return 'car_autohome_detail_'+str(df['detail_slug'])
    return df['model_detail_slug']

car_autohome_final = pd.read_csv('./man.csv')
car_autohome_final.loc[(car_autohome_final['cos_similar'] < 0.82),['liyang_detail_model','model_detail_slug','liyang_price_bn','cos_similar']] = np.NaN,np.NaN,np.NaN,np.NaN
car_autohome_final.loc[(abs(car_autohome_final['price_bn'] - car_autohome_final['liyang_price_bn']) > 1),['liyang_detail_model','model_detail_slug','liyang_price_bn','cos_similar']] = np.NaN,np.NaN,np.NaN,np.NaN
dup_detail = car_autohome_final.groupby(['model_detail_slug'])['liyang_detail_model'].count().reset_index()
dup_detail = dup_detail.loc[(dup_detail['liyang_detail_model'] > 1),'model_detail_slug'].values
car_autohome_final.loc[(car_autohome_final['model_detail_slug'].isin(dup_detail)),['liyang_detail_model','model_detail_slug','liyang_price_bn','cos_similar']] = np.NaN,np.NaN,np.NaN,np.NaN
car_autohome_final['model_detail_slug'] = car_autohome_final.apply(process_detail_slug,axis=1)
car_autohome_final = car_autohome_final.drop(['final_text','fill_vector','liyang_detail_model','liyang_price_bn','cos_similar'],axis=1)
car_autohome_final.to_csv('./man1.csv',index=False)
car_autohome_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32672 entries, 0 to 32671
Data columns (total 21 columns):
detail_slug          32672 non-null int64
brand_name           32672 non-null object
brand_slug           32672 non-null int64
model_name           32672 non-null object
model_slug           32672 non-null int64
detail_name          32672 non-null object
manufacturer         32672 non-null object
price_bn             32672 non-null float64
body                 32672 non-null object
energy               32672 non-null object
volume_extend        32672 non-null object
emission_standard    32672 non-null object
listed_year          32672 non-null int64
online_year          32672 non-null int64
volume               32672 non-null float64
control              32672 non-null object
liyang_brand_id      25869 non-null float64
liyang_model_id      32490 non-null float64
liyang_brand_slug    32672 non-null object
liyang_model_slug    32672 non-null object
model_detail_slug    32672 non-n

### 三.品牌车系款型组装

#### 1.品牌

In [163]:
def process_keywords(df):
    p = Pinyin()
    keywords = p.get_pinyin(df['name'], '')
    final_keywords = ','+keywords+','+df['name']+','+keywords
    first_letter = p.get_initial(df['name'][0])
    return pd.Series([final_keywords,first_letter,keywords])

origin_combine_brand = pd.read_csv('./origin_open_category.csv')
origin_combine_brand = origin_combine_brand.loc[(origin_combine_brand['parent'].isnull()),:].reset_index(drop=True)
origin_combine_brand = origin_combine_brand.drop(['name'],axis=1)
car_autohome_brand = car_autohome_final.loc[:,['liyang_brand_slug','brand_name','brand_slug']].rename(columns={'liyang_brand_slug':'slug','brand_slug':'car_autohome_brand_id'})
car_autohome_brand = car_autohome_brand.drop_duplicates(['slug','brand_name'])
car_autohome_brand = car_autohome_brand.merge(origin_combine_brand,how='left',on=['slug']).rename(columns={'brand_name':'name'})
car_autohome_brand[['keywords', 'first_letter', 'pinyin']] = car_autohome_brand.apply(process_keywords, axis=1)
car_autohome_brand['brand_area'] = car_autohome_brand['brand_area'].fillna('国产')
car_autohome_brand['has_detailmodel'] = 1
car_autohome_brand['units'] = 0
car_autohome_brand['score'] = 0
car_autohome_brand['status'] = 'Y'
car_autohome_brand['popular'] = 'B'
car_autohome_brand['on_sale'] = '1'
car_autohome_brand.to_csv('./new_combine_brand.csv',index=False)

temp = car_autohome_brand.loc[:,['car_autohome_brand_id']]
temp = temp.drop_duplicates(['car_autohome_brand_id'])
print('最终去重之后品牌数:',len(temp))

temp = pd.read_csv('./car_autohome_all.csv',low_memory=False)
temp = temp.loc[:,['brand_slug','brand_name']]
temp = temp.drop_duplicates(['brand_slug','brand_name'])
print('汽车之家品牌数:',len(temp))
car_autohome_brand.info()

最终去重之后品牌数: 201
汽车之家品牌数: 201
<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 200
Data columns (total 29 columns):
slug                     201 non-null object
name                     201 non-null object
car_autohome_brand_id    201 non-null int64
id                       171 non-null float64
source_id                102 non-null float64
alias                    0 non-null object
url                      98 non-null object
parent                   0 non-null object
checker_runtime_id       86 non-null float64
keywords                 201 non-null object
classified               0 non-null object
classified_url           0 non-null object
slug_global              0 non-null float64
logo_img                 129 non-null object
mum                      0 non-null object
first_letter             201 non-null object
has_detailmodel          201 non-null int64
starting_price           0 non-null float64
classified_slug          0 non-null object
thumbnail                12

#### 2.车系

In [164]:
car_autohome_all = pd.read_csv('./car_autohome_all.csv')
origin_combine_model = pd.read_csv('./origin_open_category.csv')
origin_combine_model = origin_combine_model.loc[(origin_combine_model['parent'].notnull()),:].reset_index(drop=True)
origin_combine_model = origin_combine_model.drop(['name','parent','classified','mum'],axis=1)
car_autohome_model = car_autohome_final.loc[:,['liyang_model_slug','model_name','liyang_brand_slug','body', 'manufacturer','model_slug']].rename(columns={'liyang_model_slug':'slug','liyang_brand_slug':'parent','body':'classified','manufacturer':'mum','model_slug':'car_autohome_model_id'})
car_autohome_model = car_autohome_model.drop_duplicates(['slug','model_name'])
car_autohome_model = car_autohome_model.merge(origin_combine_model,how='left',on=['slug']).rename(columns={'model_name':'name'})
car_autohome_model[['keywords', 'first_letter', 'pinyin']] = car_autohome_model.apply(process_keywords, axis=1)
car_autohome_model.loc[(car_autohome_model['attribute'].isnull()), 'attribute'] = '国产'
car_autohome_model['normalized_name'] = car_autohome_model['name']
car_autohome_model['has_detailmodel'] = 1
car_autohome_model['units'] = 0
car_autohome_model['score'] = 0
car_autohome_model['status'] = 'Y'
car_autohome_model['popular'] = 'B'
car_autohome_model['on_sale'] = '1'
car_autohome_model.to_csv('./new_combine_model.csv',index=False)
car_autohome_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1882 entries, 0 to 1881
Data columns (total 29 columns):
slug                     1882 non-null object
name                     1882 non-null object
parent                   1882 non-null object
classified               1882 non-null object
mum                      1882 non-null object
car_autohome_model_id    1882 non-null int64
id                       1853 non-null float64
source_id                923 non-null float64
alias                    42 non-null object
url                      800 non-null object
checker_runtime_id       544 non-null float64
keywords                 1882 non-null object
classified_url           561 non-null object
slug_global              0 non-null float64
logo_img                 1830 non-null object
first_letter             1882 non-null object
has_detailmodel          1882 non-null int64
starting_price           1045 non-null float64
classified_slug          1552 non-null object
thumbnail                

In [165]:
car_autohome_all = pd.read_csv('./car_autohome_all.csv')
origin_open_model_detail = pd.read_csv('./origin_open_model_detail.csv')
origin_open_model_detail = origin_open_model_detail.loc[(origin_open_model_detail['status'] != 'D'),:]
origin_open_model_detail = origin_open_model_detail.drop(['detail_model','price_bn','global_slug','year','listed_year','volume','control','emission_standard','volume_extend'],axis=1)

car_autohome_detail = car_autohome_final.loc[:,['model_detail_slug','detail_name','detail_slug','price_bn','liyang_model_slug','online_year','listed_year','volume','control','emission_standard','volume_extend']].rename(columns={'model_detail_slug':'detail_model_slug','detail_slug':'car_autohome_detail_id','detail_name':'detail_model','liyang_model_slug':'global_slug','online_year':'year'})
car_autohome_detail = car_autohome_detail.merge(origin_open_model_detail,how='left',on=['detail_model_slug']).rename(columns={'model_name':'name'})
car_autohome_detail['created_on'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
car_autohome_detail['status'] = 'Y'
car_autohome_detail['has_param'] = 'Y'
car_autohome_detail.to_csv('./new_combine_detail.csv',index=False)
car_autohome_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32672 entries, 0 to 32671
Data columns (total 26 columns):
detail_model_slug         32672 non-null object
detail_model              32672 non-null object
car_autohome_detail_id    32672 non-null int64
price_bn                  32672 non-null float64
global_slug               32672 non-null object
year                      32672 non-null int64
listed_year               32672 non-null int64
volume                    32672 non-null float64
control                   32672 non-null object
emission_standard         32672 non-null object
volume_extend             32672 non-null object
id                        28599 non-null float64
source_id                 11586 non-null float64
checker_runtime_id        10606 non-null float64
old_dmodel                13536 non-null object
cont_vprice               17355 non-null float64
url                       10606 non-null object
domain                    20848 non-null object
status                  

In [3]:
test = pd.read_csv('./new_combine_detail.csv')
test = test.loc[:,['detail_model_slug']]
test = test.drop_duplicates(['detail_model_slug'])
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32672 entries, 0 to 32671
Data columns (total 1 columns):
detail_model_slug    32672 non-null object
dtypes: object(1)
memory usage: 510.5+ KB
